# FS-PDDLVM example

$$
\begin{align}
    \frac{\partial u}{\partial t} + z_{1}u\frac{\partial u}{\partial x} - z_{0}\frac{\partial^{2}u}{\partial x^{2}} &= 0 \quad (x,t)\in[-1,1]\times(0,1] \\
    u(-1,t) &= u(1,t) = 0 \quad t\in(0,1] \\
    u(x,0) &= c\sin(\pi x) \quad x\in[-1,1]
\end{align}
$$

In [ ]:
import os
import tensorflow as tf
import tensorflow_probability as tfp
tfd = tfp.distributions
os.environ['CUDA_VISIBLE_DEVICES']='-1'

import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)

import fenics as fe
import numpy as np
import matplotlib.pyplot as plt

import time
import gc

from LFM.utilities import *


In [ ]:
Data = 'GT'
# Data = 'Ob'

if Data == 'GT':
    dirname='data-groundTruth--burgers_1000'
if Data == 'Ob':
    dirname='data-ObsData-burgers_1000'
os.makedirs(dirname, exist_ok=True)

In [ ]:
def solve_burgers(params, nx=1000, nt=200, t0=0.0, T=1.0, degree=1, plot_updates = (True, 10)):
    dt = (T-t0)/nt
    z = params['z']
    c = params['c']
    
    mesh = fe.IntervalMesh(nx,0.,1.0)
    V = fe.FunctionSpace(mesh, "CG", degree)
    xgrid = V.tabulate_dof_coordinates()[:,0]
    solutions = np.zeros((nt + 1,len(xgrid)))
    
    bc = fe.DirichletBC(V, fe.Constant(0), "on_boundary")
    
    u = fe.Function(V)
    u_prev = fe.interpolate(fe.Expression("sin(2 * {} * pi * x[0]) * sin(pi * x[0])".format(c), degree=8), V)
    solutions[0] = u_prev.vector().get_local()
    v = fe.TestFunction(V)
    
    u_half = (u + u_prev) / 2
    F = ((u - u_prev) * v * fe.dx
         + dt * z[1] * u_half * u_half.dx(0) * v * fe.dx # nonlinear term
         + dt * z[0] * u_half.dx(0) * v.dx(0) * fe.dx)   # diffusive term
    
    J = fe.derivative(F, u)
    for i in range(nt):
        fe.solve(F == 0, u, J=J, bcs=bc)
        solutions[i + 1] = u.vector().get_local()
        u_prev.assign(u)

    del mesh
    del V
    del bc
    del v, F, J
    xSln = np.copy(xgrid)
    return solutions, xSln

In [ ]:

params = {'z': np.array([0.005, 1.]), 'c':1.}
sols, xgrid = solve_burgers(params, nx=500, nt = 100)

X, T = np.meshgrid( tf.linspace(0., 1., 500+1), tf.linspace(0., 1., 100+1), indexing='ij')
plt.contourf(X, T, np.transpose(sols, axes=[1,0]), 50)
plt.colorbar()
plt.show()

In [ ]:
dimZ = 2
dimW = 1

zDist = tfd.Uniform(low=tf.constant([0.005, 0.5], shape=[dimZ]), high=tf.constant([0.01, 1.], shape=[dimZ]))
wDist = tfd.Uniform(low=tf.constant([0.5], shape=[dimW]), high=tf.constant([2.], shape=[dimW]))

nSamples  = 1_000
allTrueZs = zDist.sample(nSamples)
allTrueWs = wDist.sample(nSamples)

nx, nt = 500, 100

startTime = time.time()
X, T = np.meshgrid( tf.linspace(0., 1., nx+1), tf.linspace(0., 1., nt+1), indexing='ij')
Xr, Tr = tf.reshape(X, [-1,1]), tf.reshape(T, [-1,1])
# prepend = 'PlottingSamples_'
prepend = ''

if Data == 'GT':
    groundTruthU = []
    np.savetxt(dirname + '/' + prepend + 'allZ.dat', allTrueZs )
    np.savetxt(dirname + '/' + prepend + 'allW.dat', allTrueWs )
    XT = tf.concat([X[None,...], T[None,...]],0)
    np.save(dirname + '/' + prepend + 'FenicsMeshXT', XT, allow_pickle=False )

elif Data == 'Ob':
    numObs        = 100 
    allRandomXTs  = []
    noisySlns     = []
    np.savetxt(dirname+'/' +prepend+'allZ.dat'   , allTrueZs )
    np.savetxt(dirname+'/' +prepend+'allW.dat'   , allTrueWs )

    sigma_y = tf.constant(0.05)

PLOT = True
for i in range(allTrueZs.shape[0]):
        
    print('allTrueZs[i]', allTrueZs[i])
    print('allTrueWs[i]', allTrueWs[i,0])

    params = {'z': allTrueZs[i].numpy(), 'c': allTrueWs[i,0]}
    sols, xgrid = solve_burgers(params, nx=nx, nt=nt)
    sols = np.transpose(sols, axes=[1,0])


    if Data == 'GT':

        groundTruthU.append(np.copy(sols) )


    if Data == 'Ob':

        randomXIndex = tf.random.uniform(minval = 0, maxval = nx, shape=[numObs], dtype=tf.int32)
        randomTIndex = tf.random.uniform(minval = 0, maxval = nt, shape=[numObs], dtype=tf.int32)

        allRandomXTs.append( tf.concat([X[randomXIndex, randomTIndex][:, None], T[randomXIndex, randomTIndex][:, None]], 1) )

        subsample_sols = sols[randomXIndex, randomTIndex] 
        noisy_subsample_sols = subsample_sols + tf.random.normal(stddev=sigma_y, shape=subsample_sols.shape)
        noisySlns.append( noisy_subsample_sols )


    print('\n\nSample {}\n\n'.format(i))

    gc.collect()
    if PLOT:
        plt.contourf(X, T, sols, 50)
        plt.ylabel(r'$t$')
        plt.xlabel(r'$x$')
        plt.tight_layout()
        plt.savefig(dirname+'/'+'contourSample{i}.png')
        plt.show()  

if Data == 'GT':
    np.save(dirname+'/' +prepend+'SolutionsUGT'   , np.array(groundTruthU),  allow_pickle=False ) 


if Data == 'Ob':
    np.save(dirname+'/' +prepend+'RandomXT' , np.array(allRandomXTs),  allow_pickle=False ) 
    np.save(dirname+'/' +prepend+'NoisySolutionsObs', np.array(noisySlns),  allow_pickle=False ) 

print('time taken = ', time.time() - startTime, 's')
if PLOT: plt.show()
